In [37]:
import numpy as np

dset = np.load("/home/tliopis/Codes/exoplanets_llopis_mary_2025/data/npz_datasets/soapgpu_nst120_nsv120_5000-5050_dx2_sm3_p60_k0p1_phi0.npz", allow_pickle=True)

for key in dset.keys():
    print(key)
    if key == "metadata":
        for key in dset["metadata"].item().keys():
            print(f"  {key}: {dset['metadata'].item()[key]}")

wavegrid
template
spectra_train
spectra_val
activity_train
activity_val
time_values_train
time_values_val
v_true_train
v_true_val
metadata
  n_spectra_file: 3292
  n_spectra: 240
  n_spectra_train: 120
  n_spectra_val: 120
  n_pixels: 5000
  wavemin: 5000.0
  wavemax: 5050.0
  downscaling_factor: 2
  smooth_after_downscaling: True
  smooth_kernel_size: 3
  add_photon_noise: False
  snr_target: None
  noise_seed: None
  original_pixels: 10000
  downscaled_pixels: 5000
  batch_size: 100
  planets_periods: [60.0]
  planets_amplitudes: [0.1]
  planets_phases: [0.0]
  activity_definition: activity = spectra_pre_noise_pre_planets - template
spectra_no_activity_train
spectra_no_activity_val


In [36]:
dset["metadata"].item().keys()

dict_keys(['n_spectra_file', 'n_spectra', 'n_spectra_train', 'n_spectra_val', 'n_pixels', 'wavemin', 'wavemax', 'downscaling_factor', 'smooth_after_downscaling', 'smooth_kernel_size', 'add_photon_noise', 'snr_target', 'noise_seed', 'original_pixels', 'downscaled_pixels', 'batch_size', 'planets_periods', 'planets_amplitudes', 'planets_phases', 'activity_definition'])